In [1]:
import tensorflow as tf
import cv2
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import json

objc[85380]: Class CaptureDelegate is implemented in both /Users/dev/Sign-Language-Translator/env/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x16b6ce480) and /Users/dev/Sign-Language-Translator/env/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x283108860). One of the two will be used. Which one is undefined.
objc[85380]: Class CVWindow is implemented in both /Users/dev/Sign-Language-Translator/env/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x16b6ce4d0) and /Users/dev/Sign-Language-Translator/env/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x281470a68). One of the two will be used. Which one is undefined.
objc[85380]: Class CVView is implemented in both /Users/dev/Sign-Language-Translator/env/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x16b6ce4f8) and /Users/dev/Sign-Language-Translator/env/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x281470a90). One of the two will be used. Which

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [3]:
# https://towardsdatascience.com/accelerated-tensorflow-model-training-on-intel-mac-gpus-aa6ee691f894
with tf.device('/GPU'):
    a = tf.random.normal(shape=(2,), dtype=tf.float32)
    b = tf.nn.relu(a)

In [4]:
mp_holistic = mp.solutions.holistic # get holistic model
mp_drawing = mp.solutions.drawing_utils # get drawing tools

In [5]:
def mp_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # convert image color to RGB
    image.flags.writeable = False                  # don't show this image in feed
    results = model.process(image)                 # make holistics prediction
    image.flags.writeable = True                   # make image writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # convert back to BGR
    return image, results

In [6]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [7]:
# cap = cv2.VideoCapture(2)
# # access mp model
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     while cap.isOpened():

#         # read feed
#         ret, frame = cap.read()

#         # get holistic detections
#         image, results = mp_detection(frame, holistic)
        
#         # draw landmarks
#         draw_landmarks(image, results)

#         # show to screen
#         cv2.imshow('OpenCV Feed', image)

#         # press q to break
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()

In [8]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    return np.concatenate([pose, face, lh, rh])

In [11]:
main_path = '/Users/dev/Sign-Language-Translator/'
wlas_df = pd.read_json(main_path + 'labels/WLASL_v0.3.json')

In [12]:
with open(main_path + 'labels/WLASL_v0.3.json', 'r') as data_file:
    json_data = data_file.read()

instance_json = json.loads(json_data)

In [12]:
metadata = pd.DataFrame()
for i in range(0, len(instance_json)):
    label_ins = instance_json[i]
    gloss = label_ins['gloss']
    for i in range(0, len(label_ins['instances'])):
        label_ins['instances'][i]['gloss'] = gloss
        #bbox = label_ins['instances'][0]['bbox']
        #del label_ins['instances'][i]['bbox']
        frame = pd.Series(label_ins['instances'][i]).to_frame().T#pd.DataFrame(label_ins['instances'][i])
        #metadata['bbox'] = bbox
        metadata = pd.concat([metadata, frame])

KeyboardInterrupt: 

In [13]:
metadata.to_csv('metadata.csv', index=False)

NameError: name 'metadata' is not defined

In [15]:
metadata = pd.read_csv('metadata.csv')

In [17]:
DATA_PATH = os.path.join('demo_subset')
actions = np.array(['hello', 'love', 'thank you'])
# no_sequences = 30
# sequence_length = 30

In [22]:
count_sequences = {}
for action in actions:
    print(action)
    vid_ids = metadata[(metadata['gloss'] == action) & (metadata['split'] == 'train')]['video_id'].unique()
    #count_sequences[action] = len(vid_ids)
    print(vid_ids)
    for id in vid_ids:
        
        file = main_path + 'experiment_subset/' + str(id) + '.mp4'

        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

            cap = cv2.VideoCapture(file)

            if cap.isOpened() == False:
                print('Error opening video stream or file ' + str(id))
                #raise TypeError
                continue

            length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

            vid_path = os.path.join(DATA_PATH, action, str(id))
            os.makedirs(vid_path)

            frame_num = 0
            while cap.isOpened():

                ret, frame = cap.read()
                if not ret: break

                # get holistic detections
                image, results = mp_detection(frame, holistic)

                points = extract_keypoints(results)

                np.save(os.path.join(vid_path, str(frame_num)), points)
                
                # draw landmarks
                #draw_landmarks(image, results)

                # show video with landmarks
                #cv2.imshow('OpenCV Feed', image)

                # press q to break
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                
                frame_num += 1

            cap.release()
            cv2.destroyAllWindows()

hello
[70017 68236 27180 67755 27181 27182 27183 27184 27173]
Error opening video stream or file 70017
Error opening video stream or file 68236


OpenCV: Couldn't read video stream from file "/Users/dev/Sign-Language-Translator/experiment_subset/70017.mp4"
OpenCV: Couldn't read video stream from file "/Users/dev/Sign-Language-Translator/experiment_subset/68236.mp4"
OpenCV: Couldn't read video stream from file "/Users/dev/Sign-Language-Translator/experiment_subset/27180.mp4"
OpenCV: Couldn't read video stream from file "/Users/dev/Sign-Language-Translator/experiment_subset/67755.mp4"


Error opening video stream or file 27180
Error opening video stream or file 67755


OpenCV: Couldn't read video stream from file "/Users/dev/Sign-Language-Translator/experiment_subset/27181.mp4"
OpenCV: Couldn't read video stream from file "/Users/dev/Sign-Language-Translator/experiment_subset/27182.mp4"


Error opening video stream or file 27181
Error opening video stream or file 27182


...

OpenCV: Couldn't read video stream from file "/Users/dev/Sign-Language-Translator/experiment_subset/57655.mp4"
OpenCV: Couldn't read video stream from file "/Users/dev/Sign-Language-Translator/experiment_subset/57665.mp4"


Error opening video stream or file 57655
Error opening video stream or file 57665


OpenCV: Couldn't read video stream from file "/Users/dev/Sign-Language-Translator/experiment_subset/57666.mp4"
OpenCV: Couldn't read video stream from file "/Users/dev/Sign-Language-Translator/experiment_subset/57658.mp4"


Error opening video stream or file 57666
Error opening video stream or file 57658
Error opening video stream or file 57659


OpenCV: Couldn't read video stream from file "/Users/dev/Sign-Language-Translator/experiment_subset/57659.mp4"


In [25]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [24]:
label_map = {label:num for num, label in enumerate(os.listdir(DATA_PATH))}
print(label_map)

{'34128.mp4': 0, '27183.mp4': 1, '57666.mp4': 2, '57667.mp4': 3, '27182.mp4': 4, '34129.mp4': 5, '27180.mp4': 6, '57659.mp4': 7, '57665.mp4': 8, '57664.mp4': 9, '57658.mp4': 10, '27181.mp4': 11, '.DS_Store': 12, '27184.mp4': 13, '57663.mp4': 14, '69502.mp4': 15, '68796.mp4': 16, '68236.mp4': 17, '66598.mp4': 18, '34121.mp4': 19, '27175.mp4': 20, '57653.mp4': 21, '34134.mp4': 22, '34136.mp4': 23, '27177.mp4': 24, '34137.mp4': 25, '34123.mp4': 26, '27173.mp4': 27, '34127.mp4': 28, '34133.mp4': 29, '70017.mp4': 30, '67755.mp4': 31, '57655.mp4': 32, '27172.mp4': 33, '34118.mp4': 34, '34124.mp4': 35, '57656.mp4': 36, '57657.mp4': 37, '34125.mp4': 38, '27171.mp4': 39}


In [15]:
num_frames = 35
sequences, labels = [], []
for action in os.listdir(DATA_PATH):
    action_dir = os.path.join(DATA_PATH, action)
    for sequence in os.listdir(action_dir):
        window = []
        seq_dir = os.path.join(DATA_PATH, action, sequence)
        seq_len = len(os.listdir(seq_dir))
        start_frame = round((seq_len - num_frames) / 2)
        end_frame = start_frame + num_frames
        #print(os.listdir(seq_dir))
        #print(seq_dir)
        for frame in range(start_frame, end_frame):
            if '{}.npy'.format(frame) not in os.listdir(seq_dir):
                print(seq_dir)
                print('{}.npy'.format(frame))
                print("Error: Too many frames" )
                raise TypeError
            #print(os.path.join(seq_dir, '{}.npy'.format(frame)))
            res = np.load(os.path.join(seq_dir, '{}.npy'.format(frame)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [16]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [18]:
X_train.shape

(25, 35, 1662)

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [20]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [21]:
model = Sequential()
model.add(LSTM(32, return_sequences=True, activation='relu', input_shape=(num_frames,1662)))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(32, return_sequences=False, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [22]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [128]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
1/1 [==============================] - 4s 4s/step - loss: 1.0893 - categorical_accuracy: 0.3600
Epoch 2/2000
1/1 [==============================] - 1s 666ms/step - loss: 2.0166 - categorical_accuracy: 0.4000
Epoch 3/2000
1/1 [==============================] - 1s 649ms/step - loss: 10.6697 - categorical_accuracy: 0.3600
Epoch 4/2000
1/1 [==============================] - 1s 645ms/step - loss: 13.9119 - categorical_accuracy: 0.3200
Epoch 5/2000
1/1 [==============================] - 1s 646ms/step - loss: 6.8795 - categorical_accuracy: 0.4800
Epoch 6/2000
1/1 [==============================] - 1s 644ms/step - loss: 10.2089 - categorical_accuracy: 0.4400
Epoch 7/2000
1/1 [==============================] - 1s 680ms/step - loss: 11.5123 - categorical_accuracy: 0.3600
Epoch 8/2000
1/1 [==============================] - 1s 703ms/step - loss: 8.7678 - categorical_accuracy: 0.4800
Epoch 9/2000
1/1 [==============================] - 1s 706ms/step - loss: 6.3528 - categorical_accuracy

KeyboardInterrupt: 

In [129]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 35, 32)            216960    
                                                                 
 lstm_10 (LSTM)              (None, 35, 64)            24832     
                                                                 
 lstm_11 (LSTM)              (None, 32)                12416     
                                                                 
 dense_9 (Dense)             (None, 32)                1056      
                                                                 
 dense_10 (Dense)            (None, 16)                528       
                                                                 
 dense_11 (Dense)            (None, 3)                 51        
                                                                 
Total params: 255,843
Trainable params: 255,843
Non-tr

In [130]:
pred = model.predict(X_test)

1/1 [==============================] - 1s 508ms/step


In [131]:
pred

array([[0.05577257, 0.34980556, 0.5944219 ],
       [0.00380231, 0.731946  , 0.26425174]], dtype=float32)

In [134]:
actions[np.argmax(pred[0])]

'thank you'

In [136]:
actions[np.argmax(y_test[1])]

'love'

In [137]:
model.save('action_2.h5')

In [23]:
model.load_weights('action_2.h5')

In [26]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [25]:
yhat = model.predict(X_train)

1/1 [==============================] - 0s 494ms/step


In [27]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [28]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[16,  1],
        [ 1,  7]],

       [[17,  0],
        [ 1,  7]],

       [[14,  2],
        [ 1,  8]]], dtype=int64)

In [29]:
accuracy_score(ytrue, yhat)

0.88

In [33]:
sequence = []
sentence = []
threshold = 0.9

cap = cv2.VideoCapture(2)
# access mp model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # read feed
        ret, frame = cap.read()

        # get holistic detections
        image, results = mp_detection(frame, holistic)
        
        # draw landmarks
        draw_landmarks(image, results)

        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-num_frames:]

        if len(sequence) == num_frames:
            out = model.predict(np.expand_dims(sequence, axis=0))[0]
            #print(actions[np.argmax(out)])


        if out[np.argmax(out)] > threshold:
            if len(sentence) > 0:
                if actions[np.argmax(out)] != sentence[-1]:
                    sentence.append(actions[np.argmax(out)])
            else:
                sentence.append(actions[np.argmax(out)])

        if len(sentence) > 5:
            sentence = sentence[-5:]

        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        

        # show to screen
        cv2.imshow('OpenCV Feed', image)

        # press q to break
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 68ms/step


In [56]:
from urllib import request
KINETICS_URL = "https://raw.githubusercontent.com/deepmind/kinetics-i3d/master/data/label_map.txt"
with request.urlopen(KINETICS_URL) as obj:
  labels = [line.decode("utf-8").strip() for line in obj.readlines()]
print("Found %d labels." % len(labels))

Found 400 labels.


In [43]:
# Utilities to open video files using CV2
def crop_center_square(frame):
  y, x = frame.shape[0:2]
  min_dim = min(y, x)
  start_x = (x // 2) - (min_dim // 2)
  start_y = (y // 2) - (min_dim // 2)
  return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]

def load_video(path, max_frames=0, resize=(224, 224)):
  cap = cv2.VideoCapture(path)
  frames = []
  try:
    while True:
      ret, frame = cap.read()
      if not ret:
        break
      frame = crop_center_square(frame)
      frame = cv2.resize(frame, resize)
      frame = frame[:, :, [2, 1, 0]]
      frames.append(frame)
      
      if len(frames) == max_frames:
        break
  finally:
    cap.release()
  return np.array(frames) / 255.0



In [39]:
video_path = 'C:/Users/rbcor/Sign-Language-Translator/videos/57653.mp4'
sample_video = load_video(video_path)
sample_video.shape

(41, 224, 224, 3)

In [41]:
import tensorflow_hub as hub

i3d = hub.load("https://tfhub.dev/deepmind/i3d-kinetics-400/1").signatures['default']

In [57]:
def predict(sample_video):
    # Add a batch axis to the sample video.
    model_input = tf.constant(sample_video, dtype=tf.float32)[tf.newaxis, ...]
    
    logits = i3d(model_input)['default'][0]
    probabilities = tf.nn.softmax(logits)

    print("Top 5 actions:")
    for i in np.argsort(probabilities)[::-1][:5]:
        print(f"  {labels[i]:22}: {probabilities[i] * 100:5.2f}%")

In [58]:
predict(sample_video)

Top 5 actions:
  sign language interpreting: 78.23%
  applying cream        : 14.89%
  washing hands         :  1.23%
  taking a shower       :  0.94%
  stretching arm        :  0.85%
